In [1]:
import os
from google.colab import userdata

In [2]:
userdata = userdata.get('github')

os.environ['GITHUB_TOKEN'] = userdata

In [3]:
!git clone https://{userdata}@github.com/miguroi/sistech.git

Cloning into 'sistech'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 45 (delta 20), reused 13 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 784.05 KiB | 2.97 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [4]:
!git status

fatal: not a git repository (or any of the parent directories): .git


In [5]:
!ls

sample_data  sistech


In [6]:
cd sistech

/content/sistech


In [7]:
cd data

/content/sistech/data


In [8]:
!ls

coursera_courses_cleaned.csv   README.md
coursera_courses_cleaned.json  sistech_preprocessing.ipynb
coursera_courses.csv	       sistech_scraping.ipynb


## Setup

In [34]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re

In [35]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [36]:
df = pd.read_csv('coursera_courses.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1750 entries, 0 to 1749
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         1750 non-null   object 
 1   organization  1750 non-null   object 
 2   rating        1704 non-null   float64
 3   review_count  1750 non-null   int64  
 4   difficulty    1750 non-null   object 
 5   course_type   1750 non-null   object 
 6   duration      1750 non-null   object 
 7   skills        1749 non-null   object 
 8   url           1750 non-null   object 
 9   is_free       1750 non-null   bool   
 10  language      0 non-null      float64
 11  course_id     1750 non-null   object 
dtypes: bool(1), float64(2), int64(1), object(8)
memory usage: 152.2+ KB


In [37]:
df.head(1)

,title,organization,rating,review_count,difficulty,course_type,duration,skills,url,is_free,language,course_id
0,Google Data Analytics,Google,4.76339,170291,Beginner,Professional Certificate,THREE_TO_SIX_MONTHS,"Data Storytelling, Rmarkdown, Data Literacy, Data Visualization, Data Presentation, Data Ethics, Data Cleansing, Interactive Data Visualization, Data Validation, Ggplot2, Tableau Software, Sampling (Statistics), Presentations, Spreadsheet Software, Data Analysis, Data Visualization Software, Stakeholder Communications, LinkedIn, Interviewing Skills, Applicant Tracking Systems",/professional-certificates/google-data-analytics,False,NaN,s12n~kr43OcbTEeqeNBKhfgCLyw


In [39]:
coursera_df = df[['title', 'organization', 'rating', 'review_count', 'difficulty', 'course_type', 'duration', 'skills']].copy()
coursera_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1750 entries, 0 to 1749
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         1750 non-null   object 
 1   organization  1750 non-null   object 
 2   rating        1704 non-null   float64
 3   review_count  1750 non-null   int64  
 4   difficulty    1750 non-null   object 
 5   course_type   1750 non-null   object 
 6   duration      1750 non-null   object 
 7   skills        1749 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 109.5+ KB


In [40]:
coursera_df = coursera_df.dropna()
coursera_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1704 entries, 0 to 1749
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         1704 non-null   object 
 1   organization  1704 non-null   object 
 2   rating        1704 non-null   float64
 3   review_count  1704 non-null   int64  
 4   difficulty    1704 non-null   object 
 5   course_type   1704 non-null   object 
 6   duration      1704 non-null   object 
 7   skills        1704 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 119.8+ KB


In [41]:
coursera_df.tail()

,title,organization,rating,review_count,difficulty,course_type,duration,skills
1745,America's Unwritten Constitution,Yale University,4.812950,278,Mixed,Course,THREE_TO_SIX_MONTHS,"Case Law, Court Systems, Legal Proceedings, Legal Research, Policy Analysis, Law, Regulation, and Compliance, Political Sciences, Civil Law, Governance"
1746,Introduction to Ableton Live,Berklee,4.775382,1113,Mixed,Course,ONE_TO_FOUR_WEEKS,"File Management, Musical Composition, Music, Music Theory, Music Performance, Instrumental Music, Automation, Editing, Software Installation, Creativity"
1747,Fixing Healthcare Delivery 2.0: Advanced Lean,University of Florida,4.727273,44,Mixed,Course,ONE_TO_THREE_MONTHS,"Lean Methodologies, Patient Safety, Patient Flow, Scientific Methods, Health Systems, Health Care Administration, Continuous Quality Improvement (CQI), Health Care, Systems Analysis"
1748,Simulation and modeling of natural processes,University of Geneva,4.275000,400,Mixed,Course,ONE_TO_THREE_MONTHS,"Simulations, Python Programming, Numerical Analysis, Mathematical Modeling, Probability, Artificial Intelligence and Machine Learning (AI/ML), Visualization (Computer Graphics), Mechanics"
1749,Operations and Patient Safety for Healthcare IT Staff,Johns Hopkins University,4.844560,193,Mixed,Course,ONE_TO_FOUR_WEEKS,"Telehealth, Patient Safety, Health Technology, Medical Equipment and Technology, Help Desk Support, Technical Support, Desktop Support, Health Care, Issue Tracking, Network Support, Change Control, Technical Documentation, General Networking, Technical Communication"


## Preprocessing

In [42]:
coursera_df.head(1)

,title,organization,rating,review_count,difficulty,course_type,duration,skills
0,Google Data Analytics,Google,4.76339,170291,Beginner,Professional Certificate,THREE_TO_SIX_MONTHS,"Data Storytelling, Rmarkdown, Data Literacy, Data Visualization, Data Presentation, Data Ethics, Data Cleansing, Interactive Data Visualization, Data Validation, Ggplot2, Tableau Software, Sampling (Statistics), Presentations, Spreadsheet Software, Data Analysis, Data Visualization Software, Stakeholder Communications, LinkedIn, Interviewing Skills, Applicant Tracking Systems"


In [43]:
coursera_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1704 entries, 0 to 1749
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         1704 non-null   object 
 1   organization  1704 non-null   object 
 2   rating        1704 non-null   float64
 3   review_count  1704 non-null   int64  
 4   difficulty    1704 non-null   object 
 5   course_type   1704 non-null   object 
 6   duration      1704 non-null   object 
 7   skills        1704 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 119.8+ KB


### Lowercasing

In [44]:
def lowercasing(text):
    return text.lower()

coursera_df['title'] = coursera_df['title'].apply(lowercasing)
coursera_df['organization'] = coursera_df['organization'].apply(lowercasing)
coursera_df['difficulty'] = coursera_df['difficulty'].apply(lowercasing)
coursera_df['course_type'] = coursera_df['course_type'].apply(lowercasing)
coursera_df['duration'] = coursera_df['duration'].apply(lowercasing)
coursera_df['skills'] = coursera_df['skills'].astype(str).apply(lowercasing)

coursera_df.head(1)

,title,organization,rating,review_count,difficulty,course_type,duration,skills
0,google data analytics,google,4.76339,170291,beginner,professional certificate,three_to_six_months,"data storytelling, rmarkdown, data literacy, data visualization, data presentation, data ethics, data cleansing, interactive data visualization, data validation, ggplot2, tableau software, sampling (statistics), presentations, spreadsheet software, data analysis, data visualization software, stakeholder communications, linkedin, interviewing skills, applicant tracking systems"


### Noise Removal

In [45]:
import re

def clean_noise(text):
    text = str(text)

    text = re.sub(r'<.*?>', '', text)

    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    text = re.sub(r'#\w+', '', text)

    text = re.sub(r'[^\w\s]', '', text)

    text = re.sub(r'\d+', '', text)

    text = re.sub(r'\s+', ' ', text).strip()

    return text

coursera_df['title'] = coursera_df['title'].apply(clean_noise)
coursera_df['organization'] = coursera_df['organization'].apply(clean_noise)
coursera_df['difficulty'] = coursera_df['difficulty'].apply(clean_noise)
coursera_df['course_type'] = coursera_df['course_type'].apply(clean_noise)
coursera_df['duration'] = coursera_df['duration'].apply(clean_noise)
coursera_df['skills'] = coursera_df['skills'].apply(clean_noise)

coursera_df.head(1)

,title,organization,rating,review_count,difficulty,course_type,duration,skills
0,google data analytics,google,4.76339,170291,beginner,professional certificate,three_to_six_months,data storytelling rmarkdown data literacy data visualization data presentation data ethics data cleansing interactive data visualization data validation ggplot tableau software sampling statistics presentations spreadsheet software data analysis data visualization software stakeholder communications linkedin interviewing skills applicant tracking systems


### Compound Handling

In [46]:
import spacy
from collections import Counter, defaultdict

def detect_compound_patterns(df, text_column, min_freq=3):
    """Detect compounds using POS patterns and frequency"""

    nlp = spacy.load("en_core_web_sm")

    compound_patterns = [
        ['NOUN', 'NOUN'],           # "data science"
        ['ADJ', 'NOUN'],            # "machine learning"
        ['NOUN', 'NOUN', 'NOUN'],   # "natural language processing"
        ['ADJ', 'NOUN', 'NOUN'],    # "artificial neural networks"
        ['NOUN', 'ADP', 'NOUN'],    # "introduction to python"
        ['VERB', 'NOUN'],           # "deep learning"
    ]

    detected_sequences = defaultdict(int)
    texts = df[text_column].fillna('').astype(str)

    for doc in nlp.pipe(texts, batch_size=50):
        tokens = [token for token in doc if token.is_alpha and not token.is_space]

        # Check each pattern
        for pattern in compound_patterns:
            pattern_length = len(pattern)

            for i in range(len(tokens) - pattern_length + 1):
                sequence = tokens[i:i + pattern_length]

                # Check if POS pattern matches
                if [token.pos_ for token in sequence] == pattern:

                    # Additional filters
                    if all(not token.is_stop or token.pos_ == 'ADP' for token in sequence):

                        # Special handling for prepositions
                        if 'ADP' in pattern:
                            if sequence[1].text.lower() in ['to', 'of', 'for', 'in']:
                                phrase = ' '.join([token.text.lower() for token in sequence])
                                detected_sequences[phrase] += 1
                        else:
                            phrase = ' '.join([token.text.lower() for token in sequence])
                            detected_sequences[phrase] += 1

    # Filter by frequency and keep original format (with spaces)
    compounds = [phrase for phrase, count in detected_sequences.items() if count >= min_freq]

    return compounds, detected_sequences

In [47]:
def preserve_compounds(text):
    if pd.isna(text):
        return text

    # Split into tokens while preserving spaces
    tokens = str(text).lower().split()

    # Sort compounds by length (longest first)
    sorted_compounds = sorted(compounds, key=len, reverse=True)

    for compound in sorted_compounds:
        compound_tokens = compound.split()
        compound_length = len(compound_tokens)

        # Look for the compound pattern in consecutive tokens
        i = 0
        while i <= len(tokens) - compound_length:
            # Check if we have a match
            if tokens[i:i + compound_length] == compound_tokens:
                # Replace with underscored version
                replacement = '_'.join(compound_tokens)
                tokens = tokens[:i] + [replacement] + tokens[i + compound_length:]
                # Skip past the replacement to avoid re-processing
                i += 1
            else:
                i += 1

    return ' '.join(tokens)

In [48]:
compounds, sequences = detect_compound_patterns(coursera_df, 'skills', min_freq=5)
print("Pattern-detected compounds:", compounds[:15])

Pattern-detected compounds: ['data literacy', 'literacy data', 'data visualization', 'visualization data', 'data presentation', 'presentation data', 'data ethics', 'ethics data', 'data validation', 'tableau software', 'sampling statistics', 'spreadsheet software', 'software data', 'data analysis', 'analysis data']


In [49]:
coursera_df['title_tokenized'] = coursera_df['title'].apply(preserve_compounds)
coursera_df['skills_tokenized'] = coursera_df['skills'].apply(preserve_compounds)

In [50]:
print("\nOriginal skills:")
print(coursera_df['skills'].iloc[0])
print("\nTokenized version:")
print(coursera_df['skills_tokenized'].iloc[1])


Original skills:
data storytelling rmarkdown data literacy data visualization data presentation data ethics data cleansing interactive data visualization data validation ggplot tableau software sampling statistics presentations spreadsheet software data analysis data visualization software stakeholder communications linkedin interviewing skills applicant tracking systems

Tokenized version:
threat_modeling network security_incident response vulnerability_management computer security_incident_management hardening intrusion_detection and prevention cyber_threat_intelligence threat_management cyber_attacks cybersecurity_network protocols cloud_security vulnerability_assessments bash_scripting_language debugging linux interviewing_skills python_programming sql


In [51]:
coursera_df.head()

,title,organization,rating,review_count,difficulty,course_type,duration,skills,title_tokenized,skills_tokenized
0,google data analytics,google,4.763390,170291,beginner,professional certificate,three_to_six_months,data storytelling rmarkdown data literacy data visualization data presentation data ethics data cleansing interactive data visualization data validation ggplot tableau software sampling statistics presentations spreadsheet software data analysis data visualization software stakeholder communications linkedin interviewing skills applicant tracking systems,google data analytics,data storytelling rmarkdown data_literacy_data visualization_data_presentation data_ethics_data cleansing interactive_data_visualization data_validation ggplot tableau_software sampling_statistics presentations spreadsheet_software_data analysis_data_visualization software stakeholder_communications linkedin interviewing_skills applicant_tracking_systems
1,google cybersecurity,google,4.822850,53304,beginner,professional certificate,three_to_six_months,threat modeling network security incident response vulnerability management computer security incident management hardening intrusion detection and prevention cyber threat intelligence threat management cyber attacks cybersecurity network protocols cloud security vulnerability assessments bash scripting language debugging linux interviewing skills python programming sql,google cybersecurity,threat_modeling network security_incident response vulnerability_management computer security_incident_management hardening intrusion_detection and prevention cyber_threat_intelligence threat_management cyber_attacks cybersecurity_network protocols cloud_security vulnerability_assessments bash_scripting_language debugging linux interviewing_skills python_programming sql
2,google project management,google,4.842615,130704,beginner,professional certificate,three_to_six_months,quality management project management life cycle requirements analysis project scoping project closure project management project planning agile project management continuous improvement process project controls backlogs stakeholder communications milestones project management quality assessment team management agile methodology project documentation change management interviewing skills applicant tracking systems,google project_management,quality management_project_management life_cycle requirements_analysis project scoping project_closure project_management_project planning agile_project_management continuous_improvement_process project controls backlogs stakeholder_communications milestones_project_management quality assessment team_management agile_methodology_project documentation change_management interviewing_skills applicant_tracking_systems
3,google ai essentials,google,4.835467,2749,beginner,specialization,one_to_three_months,prompt engineering generative ai artificial intelligence and machine learning aiml large language modeling process optimization productivity software workforce development digital transformation innovation technical writing emerging technologies operational efficiency business solutions machine learning software data security critical thinking analysis data analysis data quality,google ai essentials,prompt_engineering_generative ai artificial_intelligence and machine_learning_aiml large_language_modeling process_optimization productivity_software workforce_development digital_transformation innovation technical_writing emerging_technologies operational_efficiency business solutions machine learning_software data_security critical_thinking analysis_data_analysis data_quality
4,google digital marketing ecommerce,google,4.801080,40653,beginner,professional certificate,three_to_six_months,data storytelling search engine marketing media planning social media marketing google ads email marketing social media strategy search engine optimization order fulfillment social media management performance measurement spreadshee

### Stopwords Removal

In [52]:
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [53]:
def remove_stopwords(text):
    text = str(text).lower()

    preserve_terms = {
        'ai', 'ml', 'ar', 'vr', 'ui', 'ux', 'api', 'aws', 'gcp', 'sql',
        'html', 'css', 'ios', 'android', 'react', 'node', 'vue', 'php',
        'java', 'python', 'r', 'scala', 'go', 'ruby', 'swift', 'kotlin',
        '3d', 'bi', 'etl', 'devops', 'cicd', 'ci', 'cd', 'qa', 'nlp',
        'cnn', 'rnn', 'gan', 'bert', 'gpt', 'llm', 'iot', 'erp', 'crm'
    }

    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))

    filtered_tokens = []
    for token in tokens:
        if token in preserve_terms or '_' in token:
            filtered_tokens.append(token)
        elif token not in stop_words and token.isalpha():
            if len(token) > 2:
                filtered_tokens.append(token)

    return ' '.join(filtered_tokens)

In [54]:
coursera_df['title_stopwordsized'] = coursera_df['title_tokenized'].apply(remove_stopwords)
coursera_df['skills_stopwordsized'] = coursera_df['skills_tokenized'].apply(remove_stopwords)

In [55]:
coursera_df.tail()

,title,organization,rating,review_count,difficulty,course_type,duration,skills,title_tokenized,skills_tokenized,title_stopwordsized,skills_stopwordsized
1745,americas unwritten constitution,yale university,4.812950,278,mixed,course,three_to_six_months,case law court systems legal proceedings legal research policy analysis law regulation and compliance political sciences civil law governance,americas unwritten constitution,case_law court_systems legal_proceedings legal_research policy_analysis law_regulation and compliance political_sciences civil_law governance,americas unwritten constitution,case_law court_systems legal_proceedings legal_research policy_analysis law_regulation compliance political_sciences civil_law governance
1746,introduction to ableton live,berklee,4.775382,1113,mixed,course,one_to_four_weeks,file management musical composition music music theory music performance instrumental music automation editing software installation creativity,introduction to ableton live,file_management musical_composition music music_theory music_performance instrumental_music automation editing software_installation creativity,introduction ableton live,file_management musical_composition music music_theory music_performance instrumental_music automation editing software_installation creativity
1747,fixing healthcare delivery advanced lean,university of florida,4.727273,44,mixed,course,one_to_three_months,lean methodologies patient safety patient flow scientific methods health systems health care administration continuous quality improvement cqi health care systems analysis,fixing healthcare delivery advanced lean,lean_methodologies patient_safety patient flow scientific_methods health_systems_health care administration continuous_quality_improvement cqi health_care systems_analysis,fixing healthcare delivery advanced lean,lean_methodologies patient_safety patient flow scientific_methods health_systems_health care administration continuous_quality_improvement cqi health_care systems_analysis
1748,simulation and modeling of natural processes,university of geneva,4.275000,400,mixed,course,one_to_three_months,simulations python programming numerical analysis mathematical modeling probability artificial intelligence and machine learning aiml visualization computer graphics mechanics,simulation and modeling of natural processes,simulations python_programming numerical_analysis mathematical_modeling probability artificial_intelligence and machine_learning_aiml visualization_computer_graphics mechanics,simulation modeling natural processes,simulations python_programming numerical_analysis mathematical_modeling probability artificial_intelligence machine_learning_aiml visualization_computer_graphics mechanics
1749,operations and patient safety for healthcare it staff,johns hopkins university,4.844560,193,mixed,course,one_to_four_weeks,telehealth patient safety health technology medical equipment and technology help desk support technical support desktop support health care issue tracking network support change control technical documentation general networking technical communication,operations and patient_safety for healthcare it staff,telehealth patient_safety health_technology medical_equipment and technology help desk support technical_support desktop_support health_care issue tracking network support change_control technical_documentation general networking technical_communication,operations patient_safety healthcare staff,telehealth patient_safety health_technology medical_equipment technology help desk support technical_support desktop_support health_care issue tracking network support change_control technical_documentation general networking technical_communication


### Lemmatization

In [56]:
nltk.download('wordnet')
nltk.download('omw-1.4')

def apply_lemmatization(text):
    text = str(text)
    lemmatizer = WordNetLemmatizer()

    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

    return ' '.join(lemmatized_words)

coursera_df['title_lemmatized'] = coursera_df['title_stopwordsized'].apply(apply_lemmatization)
coursera_df['skills_lemmatized'] = coursera_df['skills_stopwordsized'].apply(apply_lemmatization)

coursera_df.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,title,organization,rating,review_count,difficulty,course_type,duration,skills,title_tokenized,skills_tokenized,title_stopwordsized,skills_stopwordsized,title_lemmatized,skills_lemmatized
0,google data analytics,google,4.763390,170291,beginner,professional certificate,three_to_six_months,data storytelling rmarkdown data literacy data visualization data presentation data ethics data cleansing interactive data visualization data validation ggplot tableau software sampling statistics presentations spreadsheet software data analysis data visualization software stakeholder communications linkedin interviewing skills applicant tracking systems,google data analytics,data storytelling rmarkdown data_literacy_data visualization_data_presentation data_ethics_data cleansing interactive_data_visualization data_validation ggplot tableau_software sampling_statistics presentations spreadsheet_software_data analysis_data_visualization software stakeholder_communications linkedin interviewing_skills applicant_tracking_systems,google data analytics,data storytelling rmarkdown data_literacy_data visualization_data_presentation data_ethics_data cleansing interactive_data_visualization data_validation ggplot tableau_software sampling_statistics presentations spreadsheet_software_data analysis_data_visualization software stakeholder_communications linkedin interviewing_skills applicant_tracking_systems,google data analytics,data storytelling rmarkdown data_literacy_data visualization_data_presentation data_ethics_data cleansing interactive_data_visualization data_validation ggplot tableau_software sampling_statistics presentation spreadsheet_software_data analysis_data_visualization software stakeholder_communications linkedin interviewing_skills applicant_tracking_systems
1,google cybersecurity,google,4.822850,53304,beginner,professional certificate,three_to_six_months,threat modeling network security incident response vulnerability management computer security incident management hardening intrusion detection and prevention cyber threat intelligence threat management cyber attacks cybersecurity network protocols cloud security vulnerability assessments bash scripting language debugging linux interviewing skills python programming sql,google cybersecurity,threat_modeling network security_incident response vulnerability_management computer security_incident_management hardening intrusion_detection and prevention cyber_threat_intelligence threat_management cyber_attacks cybersecurity_network protocols cloud_security vulnerability_assessments bash_scripting_language debugging linux interviewing_skills python_programming sql,google cybersecurity,threat_modeling network security_incident response vulnerability_management computer security_incident_management hardening intrusion_detection prevention cyber_threat_intelligence threat_management cyber_attacks cybersecurity_network protocols cloud_security vulnerability_assessments bash_scripting_language debugging linux interviewing_skills python_programming sql,google cybersecurity,threat_modeling network security_incident response vulnerability_management computer security_incident_management hardening intrusion_detection prevention cyber_threat_intelligence threat_management cyber_attacks cybersecurity_network protocol cloud_security vulnerability_assessments bash_scripting_language debugging linux interviewing_skills python_programming sql
2,google project management,google,4.842615,130704,beginner,professional certificate,three_to_six_months,quality management project management life cycle requirements analysis project scoping project closure project management project planning agile project management continuous improvement process project controls backlogs stakeholder communications milestones project management quality assessment team management agile methodology project documentation change management interviewing skills applicant tracking systems,google project_management,quality management_proje

In [57]:
coursera_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1704 entries, 0 to 1749
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 1704 non-null   object 
 1   organization          1704 non-null   object 
 2   rating                1704 non-null   float64
 3   review_count          1704 non-null   int64  
 4   difficulty            1704 non-null   object 
 5   course_type           1704 non-null   object 
 6   duration              1704 non-null   object 
 7   skills                1704 non-null   object 
 8   title_tokenized       1704 non-null   object 
 9   skills_tokenized      1704 non-null   object 
 10  title_stopwordsized   1704 non-null   object 
 11  skills_stopwordsized  1704 non-null   object 
 12  title_lemmatized      1704 non-null   object 
 13  skills_lemmatized     1704 non-null   object 
dtypes: float64(1), int64(1), object(12)
memory usage: 199.7+ KB


### Pipeline

In [68]:
df_coursera = df.copy()
df_coursera.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1750 entries, 0 to 1749
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         1750 non-null   object 
 1   organization  1750 non-null   object 
 2   rating        1704 non-null   float64
 3   review_count  1750 non-null   int64  
 4   difficulty    1750 non-null   object 
 5   course_type   1750 non-null   object 
 6   duration      1750 non-null   object 
 7   skills        1749 non-null   object 
 8   url           1750 non-null   object 
 9   is_free       1750 non-null   bool   
 10  language      0 non-null      float64
 11  course_id     1750 non-null   object 
dtypes: bool(1), float64(2), int64(1), object(8)
memory usage: 152.2+ KB


In [69]:
df_coursera = df_coursera.drop('language', axis=1)
df_coursera.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1750 entries, 0 to 1749
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         1750 non-null   object 
 1   organization  1750 non-null   object 
 2   rating        1704 non-null   float64
 3   review_count  1750 non-null   int64  
 4   difficulty    1750 non-null   object 
 5   course_type   1750 non-null   object 
 6   duration      1750 non-null   object 
 7   skills        1749 non-null   object 
 8   url           1750 non-null   object 
 9   is_free       1750 non-null   bool   
 10  course_id     1750 non-null   object 
dtypes: bool(1), float64(1), int64(1), object(8)
memory usage: 138.6+ KB


In [70]:
df_coursera = df_coursera.dropna()
df_coursera.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1704 entries, 0 to 1749
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         1704 non-null   object 
 1   organization  1704 non-null   object 
 2   rating        1704 non-null   float64
 3   review_count  1704 non-null   int64  
 4   difficulty    1704 non-null   object 
 5   course_type   1704 non-null   object 
 6   duration      1704 non-null   object 
 7   skills        1704 non-null   object 
 8   url           1704 non-null   object 
 9   is_free       1704 non-null   bool   
 10  course_id     1704 non-null   object 
dtypes: bool(1), float64(1), int64(1), object(8)
memory usage: 148.1+ KB


In [71]:
def cleaning_pipeline(text):
    text = lowercasing(text)
    text = clean_noise(text)
    text = preserve_compounds(text)
    text = remove_stopwords(text)
    text = apply_lemmatization(text)
    return text

df_coursera['title'] = df_coursera['title'].apply(cleaning_pipeline)
df_coursera['skills'] = df_coursera['skills'].apply(cleaning_pipeline)

df_coursera.head()

,title,organization,rating,review_count,difficulty,course_type,duration,skills,url,is_free,course_id
0,google data analytics,Google,4.763390,170291,Beginner,Professional Certificate,THREE_TO_SIX_MONTHS,data storytelling rmarkdown data_literacy_data visualization_data_presentation data_ethics_data cleansing interactive_data_visualization data_validation ggplot tableau_software sampling_statistics presentation spreadsheet_software_data analysis_data_visualization software stakeholder_communications linkedin interviewing_skills applicant_tracking_systems,/professional-certificates/google-data-analytics,False,s12n~kr43OcbTEeqeNBKhfgCLyw
1,google cybersecurity,Google,4.822850,53304,Beginner,Professional Certificate,THREE_TO_SIX_MONTHS,threat_modeling network security_incident response vulnerability_management computer security_incident_management hardening intrusion_detection prevention cyber_threat_intelligence threat_management cyber_attacks cybersecurity_network protocol cloud_security vulnerability_assessments bash_scripting_language debugging linux interviewing_skills python_programming sql,/professional-certificates/google-cybersecurity,False,s12n~Dy6K-2UKEe2PIRJn6nL9pQ
2,google project_management,Google,4.842615,130704,Beginner,Professional Certificate,THREE_TO_SIX_MONTHS,quality management_project_management life_cycle requirements_analysis project scoping project_closure project_management_project planning agile_project_management continuous_improvement_process project control backlog stakeholder_communications milestones_project_management quality assessment team_management agile_methodology_project documentation change_management interviewing_skills applicant_tracking_systems,/professional-certificates/google-project-management,False,s12n~fq9UWMbTEeqpthJ2RmWGow
3,google ai essential,Google,4.835467,2749,Beginner,Specialization,ONE_TO_THREE_MONTHS,prompt_engineering_generative ai artificial_intelligence machine_learning_aiml large_language_modeling process_optimization productivity_software workforce_development digital_transformation innovation technical_writing emerging_technologies operational_efficiency business solution machine learning_software data_security critical_thinking analysis_data_analysis data_quality,/specializations/ai-essentials-google,False,s12n~3tzIujTqTk-YdcEFZ9r3sQ
4,google digital_marketing ecommerce,Google,4.801080,40653,Beginner,Professional Certificate,THREE_TO_SIX_MONTHS,data storytelling search_engine_marketing medium planning social_media_marketing google ad email_marketing social_media_strategy search_engine_optimization order fulfillment social_media management_performance_measurement spreadsheet_software testing customer_retention ecommerce campaign_management loyalty program marketing interviewing_skills applicant_tracking_systems,/professional-certificates/google-digital-marketing-ecommerce,False,s12n~aYYrIEl-EeyCjQ5Y8Mzdsw


In [72]:
df_coursera.tail()

,title,organization,rating,review_count,difficulty,course_type,duration,skills,url,is_free,course_id
1745,america unwritten constitution,Yale University,4.812950,278,Mixed,Course,THREE_TO_SIX_MONTHS,case_law court_systems legal_proceeding legal_research policy_analysis law_regulation compliance political_science civil_law governance,/learn/unwritten-constitution,True,course~aAo9tS8NEeWv_w7cMMH1Uw
1746,introduction ableton live,Berklee,4.775382,1113,Mixed,Course,ONE_TO_FOUR_WEEKS,file_management musical_composition music music_theory music_performance instrumental_music automation editing software_installation creativity,/learn/ableton-live,False,course~-nRRAPmuEeSndSIAC7LTLQ
1747,fixing healthcare delivery advanced lean,University of Florida,4.727273,44,Mixed,Course,ONE_TO_THREE_MONTHS,lean_methodologies patient_safety patient flow scientific_method health_systems_health care administration continuous_quality_improvement cqi health_care systems_analysis,/learn/fixing-healthcare-delivery-advanced-lean,False,course~7XFCjCE3EeeIYhLE7kr1cg
1748,simulation modeling natural process,University of Geneva,4.275000,400,Mixed,Course,ONE_TO_THREE_MONTHS,simulation python_programming numerical_analysis mathematical_modeling probability artificial_intelligence machine_learning_aiml visualization_computer_graphics mechanic,/learn/modeling-simulation-natural-processes,True,course~qwwhTCF_EeWXzxJxfIL00w
1749,operation patient_safety healthcare staff,Johns Hopkins University,4.844560,193,Mixed,Course,ONE_TO_FOUR_WEEKS,telehealth patient_safety health_technology medical_equipment technology help desk support technical_support desktop_support health_care issue tracking network support change_control technical_documentation general networking technical_communication,/learn/healthcare-it-operations-patient-safety,False,course~z7bOth-eEeuQ9xLSu4r6yQ


## Final Dataset

In [73]:
df_coursera_csv = 'coursera_courses_cleaned.csv'
df_coursera.to_csv(df_coursera_csv, index=False)

print(f"Data successfully saved to file: '{df_coursera_csv}'")

Data successfully saved to file: 'coursera_courses_cleaned.csv'


In [74]:
df_coursera_json = 'coursera_courses_cleaned.json'
df_coursera.to_json(df_coursera_json, orient='records', indent=4)

print(f"Data successfully saved to file: '{df_coursera_json}'")

Data successfully saved to file: 'coursera_courses_cleaned.json'
